We have noticed there are many rows in new transaction data in which merchant id is blank.
We have also noticed that for the combination of 'merchant_category_id','state_id','city_id','subsector_id','category_1'
there are no unique merchant id that we can find in merchants data

Now considering that merchant id is string value, and will not end up in the final model data set,we are going to do below:

1. group merchants data by 'merchant_category_id','state_id','city_id','subsector_id','category_1'
2. take aggregate on the other columns 
3. keep that data ready to be joined with new transaction data and historical transactions


Note: the merchant data that we are taking here is a feature engineered merchant data where we have already filled in missing data , and we have also performed some aggregations and performed one hot encoding on categorical data

In [24]:
import pandas as panda
import numpy as np
from collections import Counter

In [2]:
new_transaction_path = 'C:/Users/somak/Documents/somak_python/real_world_use_cases_1/supervised/regression/kaggle/elo/all/new_merchant_transactions.csv'
merchants_data_path = 'all_new/merchants_1.csv'

In [18]:
# new_transaction_data = panda.read_csv(new_transaction_path)
merchant_data = panda.read_csv(merchants_data_path)

In [21]:

## check if null values are present in any of the group by columns 'merchant_category_id','state_id','city_id','subsector_id','category_1'
merchant_data[['merchant_category_id','state_id','city_id','subsector_id','category_1']].isnull().any()

merchant_category_id    False
state_id                False
city_id                 False
subsector_id            False
category_1              False
dtype: bool

In [44]:
merchant_data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
merchant_id,334696,334633,M_ID_dbbf07ebf0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
merchant_group_id,334696,NaN,NaN,NaN,31028.7,31623,1,3612,19900,51707.2,112586
merchant_category_id,334696,NaN,NaN,NaN,423.132,252.898,-1,222,373,683,891
subsector_id,334696,NaN,NaN,NaN,25.1165,9.80727,1,19,27,33,41
numerical_1,334696,NaN,NaN,NaN,0.0114764,1.09815,-0.0574706,-0.0574706,-0.0574706,-0.0475558,183.735
numerical_2,334696,NaN,NaN,NaN,0.00810311,1.0705,-0.0574706,-0.0574706,-0.0574706,-0.0475558,182.079
category_1,334696,2,N,327657,NaN,NaN,NaN,NaN,NaN,NaN,NaN
avg_sales_lag3,334696,NaN,NaN,NaN,13.833,2395.44,-82.13,0.88,1,1.16,851845
avg_purchases_lag3,334696,NaN,NaN,NaN,2.14514,213.956,0.333495,0.92365,1.01667,1.14652,61851.3
active_months_lag3,334696,NaN,NaN,NaN,2.99411,0.0952475,1,3,3,3,3


In [23]:
merchant_dtypes = merchant_data.dtypes.to_frame().reset_index()
merchant_dtypes.columns = ['col_name','col_type']
merchant_dtypes

,col_name,col_type
0,merchant_id,object
1,merchant_group_id,int64
2,merchant_category_id,int64
3,subsector_id,int64
4,numerical_1,float64
5,numerical_2,float64
6,category_1,object
7,avg_sales_lag3,float64
8,avg_purchases_lag3,float64
9,active_months_lag3,int64


In [37]:
merchant_data.avg_purchases_lag3.isnull().sum(),\
merchant_data[merchant_data.avg_purchases_lag3==np.inf].avg_purchases_lag3.any(),\
merchant_data[merchant_data.avg_purchases_lag6==np.inf].avg_purchases_lag6.any(),\
merchant_data[merchant_data.avg_purchases_lag12==np.inf].avg_purchases_lag12.any()

(0, True, True, True)

In [41]:
## replacing infinite numbers with max of avg purchase sales
max_avg_purchase_3 = merchant_data[merchant_data.avg_purchases_lag3!=np.inf].avg_purchases_lag3.max()
max_avg_purchase_6 = merchant_data[merchant_data.avg_purchases_lag6!=np.inf].avg_purchases_lag6.max()
max_avg_purchase_12 = merchant_data[merchant_data.avg_purchases_lag12!=np.inf].avg_purchases_lag12.max()
max_avg_purchase_3,max_avg_purchase_6,max_avg_purchase_12

(61851.33333333, 56077.5, 50215.55555556)

In [43]:
merchant_data.avg_purchases_lag3.replace(np.inf, max_avg_purchase_3, inplace=True)
merchant_data.avg_purchases_lag6.replace(np.inf, max_avg_purchase_6, inplace=True)
merchant_data.avg_purchases_lag12.replace(np.inf, max_avg_purchase_12, inplace=True)
merchant_data[merchant_data.avg_purchases_lag3==np.inf].avg_purchases_lag3.any(),\
merchant_data[merchant_data.avg_purchases_lag6==np.inf].avg_purchases_lag6.any(),\
merchant_data[merchant_data.avg_purchases_lag12==np.inf].avg_purchases_lag12.any()

(False, False, False)

In [91]:
def most_frequent_merchant_id(x):
    c = Counter(x)
    return c.most_common()[0][0]

def most_frequent_merchant_group_id(x):
    c = Counter(x)
    return c.most_common()[0][0]


aggregate ={
    
    
    'merchant_group_id':['nunique'],
    'merchant_id':['nunique',most_frequent_merchant_id],
    'numerical_1'      :['sum','mean','max','min'],
    'numerical_2'      :['sum','mean','max','min'],
    'avg_sales_lag3'   :['sum','mean','max','min'],
    'avg_purchases_lag3':['sum','mean','max','min'],
    'active_months_lag3':['sum','mean','max','min'],
    'avg_sales_lag6':['sum','mean','max','min'],
    'avg_purchases_lag6':['sum','mean','max','min'],
    'active_months_lag6':['sum','mean','max','min'],
    'avg_sales_lag12':['sum','mean','max','min'],
    'avg_purchases_lag12':['sum','mean','max','min'],
    'active_months_lag12':['sum','mean','max','min'],
    'merchant_category_1_yes':['sum'],
    'merchant_category_1_no':['sum'],
    'merchant_category_4_yes':['sum'],
    'merchant_category_4_no':['sum'],
    'most_sales_range_E':['sum'],
    'most_sales_range_D':['sum'],
    'most_sales_range_C':['sum'],
    'most_sales_range_B':['sum'],
    'most_sales_range_A':['sum'],
    'most_recent_purchases_range_E':['sum'],
    'most_recent_purchases_range_D':['sum'],
    'most_recent_purchases_range_C':['sum'],
    'most_recent_purchases_range_B':['sum'],
    'most_recent_purchases_range_A':['sum'],
    
    
    
}
    

In [92]:
t = merchant_data.groupby(['merchant_category_id','state_id','city_id','subsector_id','category_1']).agg(aggregate)

In [93]:
t.reset_index(inplace = True)
t.head()

merchant_category_id state_id city_id subsector_id category_1  \
                                                                  
0                   -1        9     231           27          N   
1                    2        1      -1           20          N   
2                    2        1     246           20          N   
3                    2        2      -1           20          N   
4                    2        2      76           20          N   

  merchant_group_id merchant_id                           numerical_1  \
            nunique     nunique most_frequent_merchant_id         sum   
0                 1           1           M_ID_a2cfe4149a   -0.047556   
1                 4           4           M_ID_eeba19250e    0.037820   
2                 4           4           M_ID_628321e5da    0.027905   
3                 4           4           M_ID_0284e321ff    0.672374   
4                15          15           M_ID_6ef27dc5bd   -0.148187   

                         ...              most_sales_range_E  \
       mean              ...                             sum   
0 -0.047556              ...                               1   
1  0.009455              ...                               1   
2  0.006976              ...                               1   
3  0.168093              ...                               3   
4 -0.009879              ...                               4   

  most_sales_range_D most_sales_range_C most_sales_range_B most_sales_range_A  \
                 sum                sum                sum                sum   
0                  0                  0                  0                  0   
1                  3                  0                  0                  0   
2                  3                  0                  0                  0   
3                  0                  1                  0                  0   
4                  7                  4                  0                  0   

  most_recent_purchases_range_E most_recent_purchases_range_D  \
                            sum                           sum   
0                             1                             0   
1                             2                             2   
2                             2                             2   
3                             3                             1   
4                             4                            11   

  most_recent_purchases_range_C most_recent_purchases_range_B  \
                            sum                           sum   
0                             0                             0   
1                             0                             0   
2                             0                             0   
3                             0                             0   
4                             0                             0   

  most_recent_purchases_range_A  
                            sum  
0                             0  
1                             0  
2                             0  
3                             0  
4                             0  

[5 rows x 66 columns]

In [94]:
t.columns = ['_'.join(i) for i in t.columns.tolist()]
t.head()

,merchant_category_id_,state_id_,city_id_,subsector_id_,category_1_,merchant_group_id_nunique,merchant_id_nunique,merchant_id_most_frequent_merchant_id,numerical_1_sum,numerical_1_mean,...,most_sales_range_E_sum,most_sales_range_D_sum,most_sales_range_C_sum,most_sales_range_B_sum,most_sales_range_A_sum,most_recent_purchases_range_E_sum,most_recent_purchases_range_D_sum,most_recent_purchases_range_C_sum,most_recent_purchases_range_B_sum,most_recent_purchases_range_A_sum
0,-1,9,231,27,N,1,1,M_ID_a2cfe4149a,-0.047556,-0.047556,...,1,0,0,0,0,1,0,0,0,0
1,2,1,-1,20,N,4,4,M_ID_eeba19250e,0.037820,0.009455,...,1,3,0,0,0,2,2,0,0,0
2,2,1,246,20,N,4,4,M_ID_628321e5da,0.027905,0.006976,...,1,3,0,0,0,2,2,0,0,0
3,2,2,-1,20,N,4,4,M_ID_0284e321ff,0.672374,0.168093,...,3,0,1,0,0,3,1,0,0,0
4,2,2,76,20,N,15,15,M_ID_6ef27dc5bd,-0.148187,-0.009879,...,4,7,4,0,0,4,11,0,0,0


In [95]:
t.columns = list(map(lambda x: x[:len(x)-1] if x.endswith('_') else x, t.columns.tolist()))

In [96]:
t.head()

,merchant_category_id,state_id,city_id,subsector_id,category_1,merchant_group_id_nunique,merchant_id_nunique,merchant_id_most_frequent_merchant_id,numerical_1_sum,numerical_1_mean,...,most_sales_range_E_sum,most_sales_range_D_sum,most_sales_range_C_sum,most_sales_range_B_sum,most_sales_range_A_sum,most_recent_purchases_range_E_sum,most_recent_purchases_range_D_sum,most_recent_purchases_range_C_sum,most_recent_purchases_range_B_sum,most_recent_purchases_range_A_sum
0,-1,9,231,27,N,1,1,M_ID_a2cfe4149a,-0.047556,-0.047556,...,1,0,0,0,0,1,0,0,0,0
1,2,1,-1,20,N,4,4,M_ID_eeba19250e,0.037820,0.009455,...,1,3,0,0,0,2,2,0,0,0
2,2,1,246,20,N,4,4,M_ID_628321e5da,0.027905,0.006976,...,1,3,0,0,0,2,2,0,0,0
3,2,2,-1,20,N,4,4,M_ID_0284e321ff,0.672374,0.168093,...,3,0,1,0,0,3,1,0,0,0
4,2,2,76,20,N,15,15,M_ID_6ef27dc5bd,-0.148187,-0.009879,...,4,7,4,0,0,4,11,0,0,0


In [97]:
t.describe(include='all').T.head(40)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
merchant_category_id,46421,NaN,NaN,NaN,434.293,244.823,-1,225,420,645,891
state_id,46421,NaN,NaN,NaN,12.5006,5.75688,1,9,12,16,24
city_id,46421,NaN,NaN,NaN,152.302,108.765,-1,51,153,248,347
subsector_id,46421,NaN,NaN,NaN,21.7868,10.7581,1,12,25,29,41
category_1,46421,2,N,46154,NaN,NaN,NaN,NaN,NaN,NaN,NaN
merchant_group_id_nunique,46421,NaN,NaN,NaN,7.10594,37.0483,1,1,2,4,2794
merchant_id_nunique,46421,NaN,NaN,NaN,7.20868,37.6229,1,1,2,4,2886
merchant_id_most_frequent_merchant_id,46421,46421,M_ID_816e6ced72,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
numerical_1_sum,46421,NaN,NaN,NaN,0.0827448,2.13859,-82.4981,-0.0831778,-0.045537,0.0218486,139.487
numerical_1_mean,46421,NaN,NaN,NaN,0.00550269,0.135858,-0.0574707,-0.0475558,-0.0268997,0.0119337,9.10886


In [98]:
t.describe(include='all').T.tail(40)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
active_months_lag3_max,46421,NaN,NaN,NaN,2.99582,0.0805503,1,3,3,3,3
active_months_lag3_min,46421,NaN,NaN,NaN,2.96437,0.234978,1,3,3,3,3
avg_sales_lag6_sum,46421,NaN,NaN,NaN,156.102,10605.6,-30.38,1.18,2.35,6.4,1.51396e+06
avg_sales_lag6_mean,46421,NaN,NaN,NaN,24.5593,2115.54,-0.660435,0.955,1.15,1.53813,378491
avg_sales_lag6_max,46421,NaN,NaN,NaN,146.887,10597.2,0.19,1.05,1.37,2.35,1.51396e+06
avg_sales_lag6_min,46421,NaN,NaN,NaN,4.3755,486.399,-82.13,0.67,0.89,1.15,93735
avg_purchases_lag6_sum,46421,NaN,NaN,NaN,17.2333,529.433,0.1766,1.15909,2.25,5.95002,56172.9
avg_purchases_lag6_mean,46421,NaN,NaN,NaN,1.85686,35.8647,0.1766,0.983333,1.13537,1.41667,7010.77
avg_purchases_lag6_max,46421,NaN,NaN,NaN,9.21547,521.529,0.1766,1.05753,1.30667,2,56077.5
avg_purchases_lag6_min,46421,NaN,NaN,NaN,1.17943,10.0083,0.167045,0.754902,0.932222,1.13686,2113.17


In [99]:
temp=t.isnull().sum().to_frame().reset_index()
temp.columns = ['col_name','count_']
temp.loc[temp.count_>0]

,col_name,count_


In [100]:
t.shape, merchant_data.shape


((46421, 66), (334696, 34))

In [101]:
t.to_csv('all_new/agg_merchants_1.csv', index=False)